# NOTÍCIAS 2022

Planilha salva no Google Sheets:

https://docs.google.com/spreadsheets/d/1X2rxOAnLpC6gwFhm16wWkA-YvDYpq6l137BJb7dFHMM/edit#gid=1484081685


Bibliotecas:
    - BeautifulSoup
    - Pandas
    - openpyxl
    - copy
    - reduce
    
1. Extrair links das notícias com Selenium:
    - links2022.xlsx
    
2. Importar links das notícias (para o caso de editar no TRE-SP e não ter acesso ao Selenium):
    
3. Acessar links e extrair informações, armazenadas em uma lista de dicionários para conversão em dataframe:
    - data
    - theme (dicionario vazio)
    - tag
    - title
    - channel (dicionario 'Site')
    - tempo de leitura
    - horário de publicação
    - subtítulo
    - conteúdo
    - lista com as notícias relacionadas (mesmo modelo das tags)

4. Gerar planilha com as informações

### Bibliotecas

In [2]:
from urllib.request import Request, urlopen, urlretrieve
from urllib.error import URLError, HTTPError
from bs4 import BeautifulSoup
import pandas as pd
import openpyxl
import copy
from functools import reduce
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

servico = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=servico)

### 1. Extração dos links das notícias

In [ ]:
# URL para o ano de 2022:
driver.get('https://www.tre-sp.jus.br/comunicacao/noticias?year=2022&month=&period=&formData=')

#iterador para rolar a página de notícias 
for i in range(200):
    try:
        driver.find_element(By.XPATH, '//*[@id="news-list-read-more-button"]').click()
        time.sleep(10)
    except:
        pass

#lista de notícias 

news = driver.find_elements(By.CLASS_NAME, 'news-list-title')

# variavel para iterar com for e pegar links

list_links = []
for n in news:
    link = n.find_element(By.TAG_NAME, 'a').get_attribute('href')
    list_links.append(link)
    
# dicionário para criar dataframe com os links e, em seguida, exportar para excel
df_list_links = pd.DataFrame({'LINK': list_links})
df_list_links.to_excel('data/links2022.xlsx', index=False)


### 2. Importar links das notícias (para o caso de editar no TRE-SP e não ter acesso ao Selenium)

In [3]:
links = pd.read_excel('data/links2022.xlsx')

### 3. Acessar links e extrair informações

In [5]:
list_dict_links = []
dict_to_df = {}

for l in links['LINK']:

    response = urlopen(l)
    html = response.read().decode('utf-8')
    soup = BeautifulSoup(html, 'html.parser')

    dict_link = {}
    
    #data
    dict_link['date'] = soup.find('div', {"class": "data-hora"}).get_text().split()[0]
    dict_to_df['DATA'] = dict_link['date']
    
    #hora
    dict_link['hour'] = soup.find('div', {"class": "data-hora"}).get_text().split()[1]
    dict_to_df['HORA'] = dict_link['hour']

    #título
    title = soup.find('h2', {"class": "noticia-titulo"})
    dict_link['title'] = soup.find('h2', {"class": "noticia-titulo"}).get_text()
    dict_to_df['TÍTULO DA MATÉRIA'] = dict_link['title']

    #subtítulo
    dict_link['subtitle'] = title.findNext().get_text()
    dict_to_df['SUBTÍTULO'] = dict_link['subtitle']

    #assunto/tema
    dict_link['theme'] = None
    dict_to_df['TEMA'] = dict_link['theme']

    #tags
    tags = []
    tags_name = soup.findAll('li', {"class": "tag"})
    for i in tags_name:
        tag_text = i.findNext().get('title').replace('Link: ', '')
        tags.append(tag_text)

    tags_list = (tags)
    tc = reduce(lambda x, y: x + ' , ' + y, tags_list)

    dict_link['tags'] = tc
    dict_to_df['TAGS'] = dict_link['tags']

    #tempo de leitura:
    time = soup.find('div', {"class": "tempo-leitura"}).get_text().split()

    for i in time:
        try:
            time.remove('de')
            time.remove('leitura')
        except:
            pass


    #tempo de leitura: removendo caracteres e convertendo para número

    chars = ['m','s']

    chars = set(chars)

    list_time = []

    for t in time:
        if t[2] == 'm':
            m = int(''.join(c for c in t if c not in chars))
            minutes = m * 60
            list_time.append(minutes)
        if t[2] == 's':
            seg = int(''.join(c for c in t if c not in chars))
            list_time.append(seg)

    reading_time = sum(list_time)

    dict_link['reading time'] = reading_time
    dict_to_df['TEMPO DE LEITURA'] = dict_link['reading time'] 
    

    #link
    dict_link['link'] = l
    dict_to_df['LINK'] = l

    #conteúdo - PROCURAR UMA MANEIRA DE FORMATAR
    dict_link['content'] = soup.find('div', {"class": "tab-pane active"}).get_text()
    dict_to_df['CONTEÚDO'] = dict_link['content']


    list_dict_links.append(dict_link)

### 4. Gerar planilha com as notícias

In [10]:
df = pd.DataFrame(list_dict_links)
df.to_excel('output/scraping_noticias_2022.xlsx')

In [2]:
teste = 'teste'

In [ ]:
teste_2 = 'teste_2'